# Biomarkers and metabolomics in the (Geenen, 2012) model

This is for advanced students or student projects. 

<span style="color:red">**Assignment:**</span> Read the (Geenen, 2012) publication. 

<span style="color:red">**Assignment:**</span> The teaching assistants can supply you with the kinetic model that works in Copasi. Try reproducing some of the results shown in the publication. Especially, the relationship between paracetamol increase and ophthalmic acid and oxoproline. 

<span style="color:red">**Assignment:**</span> What, if anything, can we say about these biomarkers with FBA? Specifically, given the method by Shlomi et al?

In [ ]:
import cobra
from cobra.solvers import get_solver_name
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import parsimonious
import pandas as pd
from utils import show_map, findBiomarkers

In [ ]:
# set escher map
map_loc = './maps/escher_map_geenen_2012.json'
M = cobra.io.load_json_model('./models/Geenen_cobra_model.json')

M.reactions.EX_v_v18.lower_bound = -1 # glut
M.reactions.EX_v_v20.lower_bound = -1 # gly
M.reactions.EX_v_v39.lower_bound = -1 # met
M.reactions.EX_v_v41.lower_bound = -1 # bcys
M.reactions.EX_v_v32.lower_bound = 0 # OPA
M.reactions.EX_v_v38.lower_bound = 0 # OXO
M.reactions.EX_v_v37.lower_bound = 0 # cysASG
M.reactions.EX_v_v22.lower_bound = 0 # CH2THF

exchanges = [rxn.id for rxn in M.reactions if rxn.products == [] or rxn.reactants == []]

model = M.copy()

## Analyze basic flux distributions

When glutamate, glycine, methionine and cysteine are present in the medium paracetamol can be metabolized to an amount equal to the sum of input methionine and cysteine. The glutamate and glycine are recycled in a catabolism loop. 

In [ ]:
model = M.copy()

model.reactions.EX_para.lower_bound = -1000; model.reactions.EX_para.upper_bound = -20
model.reactions.EX_v_v41.lower_bound = -10; model.reactions.EX_v_v41.upper_bound = 1000 # cys
model.reactions.EX_v_v39.lower_bound = -10; model.reactions.EX_v_v39.upper_bound = 1000 # met
model.reactions.EX_v_v18.lower_bound = -10; model.reactions.EX_v_v18.upper_bound = 1000 # glut
model.reactions.EX_v_v20.lower_bound = -10; model.reactions.EX_v_v20.upper_bound = 1000 # gly
# model.reactions.EX_v_v38.lower_bound = -10; model.reactions.EX_v_v38.upper_bound = 100 # oxo
# model.reactions.EX_v_v32.lower_bound = -10; model.reactions.EX_v_v39.upper_bound = 100 # opa

sol = cobra.flux_analysis.parsimonious.optimize_minimal_flux(model)
b = show_map(sol,map_loc)
b.save_html('./predictions/FBA_glu-gly-met-cys_loop.html',overwrite=True)
b.display_in_notebook()

Cysteine and glutamate are not needed now because of the loop with reactions 31 and 24

<span style="color:red">**Assignment:**</span> Can you stop this loop while producing cysASG?

<span style="color:red">**Assignment:**</span> What happens when we ask for ASG directly? 

## Biomarker predictions using Shlomi et al.'s method
<span style="color:red">**Assignment:**</span> Use the findBiomarker function to predict changes in extracellular metabolite levels upon paracetamol influx. How does this depend on the medium? Specifically look at the oxoproline and ophthalmic acid predictions. Do they agree with the kinetic model? 

## Metabolomics predictor
<span style="color:red">**Assignment:**</span> Use the metabolomics parameter of findBiomarkers to predict metabolomics changes when varying paracetamol input (i.e. drug mode).

### Internal bounds lower than external bounds
Does this help?

<span style="color:red">**Assignment:**</span> Experiment with setting internal reactions bounds lower than external influxes of the virtual reactions. Does this change your results?